In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pickle


In [2]:
cd '/Users/anmolika/Downloads/MRS'

/Users/anmolika/Downloads/MRS


In [3]:
data=pd.read_excel("movie_data.xlsx")
data.info()
data.rename(columns={'Unnamed: 0': 'movie_id'}, inplace=True)

columns=['Cast','Director','Genre','Title','Description']


data[columns].isnull().values.any()#no null values



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          1000 non-null   int64  
 1   Title               1000 non-null   object 
 2   Year of Release     997 non-null    object 
 3   Watch Time          951 non-null    object 
 4   Genre               1000 non-null   object 
 5   Movie Rating        942 non-null    float64
 6   Metascore of movie  910 non-null    float64
 7   Director            1000 non-null   object 
 8   Cast                1000 non-null   object 
 9   Votes               942 non-null    object 
 10  Description         1000 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 86.1+ KB


False

In [4]:
def get_important_features(data):
    important_features=[]
    for i in range (0,data.shape[0]):
        important_features.append(data['Title'][i]+' '+data['Director'][i]+' '+data['Genre'][i]+' '+data['Description'][i])
    return important_features

In [5]:
#creating a column to hold the combined strings
data['important_features']=get_important_features(data)

In [6]:
tfidf = TfidfVectorizer(stop_words='english')
#data['Description'] = data['Description'].fillna('')
tfidf_matrix = tfidf.fit_transform(data['important_features'])
tfidf_matrix.shape

(1000, 6763)

In [7]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [9]:
indices = pd.Series(data.index, index=data['Title']).drop_duplicates()
#indices['Stillwater']
#sim_scores = list(enumerate(cosine_sim[indices['Stillwater']]))
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    # Return the top 5 most similar movies
    movies=data['Title'].iloc[movie_indices]
    id=data['movie_id'].iloc[movie_indices]
    dict={"Movies":movies,"id":id}
    final_df=pd.DataFrame(dict)
    final_df.reset_index(drop=True,inplace=True)
    return final_df


get_recommendations('Titanic')
#Stillwater
#get_recommendations('Stillwater')

,Movies,id
0,Call Me by Your Name,267
1,The Notebook,350
2,Terminator 2: Judgment Day,330
3,Lady Bird,547
4,Twilight,246


In [10]:
data.info()
new = data.drop(columns=['Year of Release','Watch Time','Genre','Movie Rating','Metascore of movie','Director','Cast','Votes','Description'])



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            1000 non-null   int64  
 1   Title               1000 non-null   object 
 2   Year of Release     997 non-null    object 
 3   Watch Time          951 non-null    object 
 4   Genre               1000 non-null   object 
 5   Movie Rating        942 non-null    float64
 6   Metascore of movie  910 non-null    float64
 7   Director            1000 non-null   object 
 8   Cast                1000 non-null   object 
 9   Votes               942 non-null    object 
 10  Description         1000 non-null   object 
 11  important_features  1000 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 93.9+ KB
